<a href="https://colab.research.google.com/github/josejailson/charRNN/blob/main/charRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the data

In [ ]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [ ]:
encoded -= 2

In [ ]:
n_tokens = text_vec_layer.vocabulary_size() - 2

In [ ]:
dataset_size = len(encoded)

# Preparing the data

In [ ]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
  ds = tf.data.Dataset.from_tensor_slices(sequence)
  ds = ds.window(length + 1, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda window_ds: window_ds.batch(length+1))
  if shuffle:
    ds = ds.shuffle(buffer_size=100_000, seed=seed)
  ds = ds.batch(batch_size)
  return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

O que to_dataset faz é pegar o encoded text e criar um dataset contendo todas as windows com o tamanho desejado.

In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set =  to_dataset(encoded[1_060_000:], length=length)

# Building and training the Char-RNN Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint("my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])

Epoch 1/10
  31247/Unknown - 1736s 54ms/step - loss: 1.4087 - accuracy: 0.5689

31247/31247 [==============================] - 1790s 55ms/step - loss: 1.4087 - accuracy: 0.5689 - val_loss: 1.6171 - val_accuracy: 0.5295
Epoch 2/10
31247/31247 [==============================] - ETA: 0s - loss: 1.3092 - accuracy: 0.5918

31247/31247 [==============================] - 1355s 42ms/step - loss: 1.3092 - accuracy: 0.5918 - val_loss: 1.5937 - val_accuracy: 0.5359
Epoch 3/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2843 - accuracy: 0.5981

31247/31247 [==============================] - 1300s 40ms/step - loss: 1.2843 - accuracy: 0.5981 - val_loss: 1.5817 - val_accuracy: 0.5408
Epoch 4/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2702 - accuracy: 0.6017

31247/31247 [==============================] - 1415s 44ms/step - loss: 1.2702 - accuracy: 0.6017 - val_loss: 1.5633 - val_accuracy: 0.5442
Epoch 5/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2626 - accuracy: 0.6034

31247/31247 [==============================] - 1975s 61ms/step - loss: 1.2626 - accuracy: 0.6034 - val_loss: 1.5651 - val_accuracy: 0.5463
Epoch 6/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2574 - accuracy: 0.6045

31247/31247 [==============================] - 1322s 41ms/step - loss: 1.2574 - accuracy: 0.6045 - val_loss: 1.5580 - val_accuracy: 0.5488
Epoch 7/10
31246/31247 [============================>.] - ETA: 0s - loss: 1.2538 - accuracy: 0.6054

31247/31247 [==============================] - 1291s 40ms/step - loss: 1.2538 - accuracy: 0.6054 - val_loss: 1.5589 - val_accuracy: 0.5501
Epoch 8/10
31247/31247 [==============================] - 1356s 42ms/step - loss: 1.2508 - accuracy: 0.6063 - val_loss: 1.5625 - val_accuracy: 0.5477
Epoch 9/10
31247/31247 [==============================] - 1391s 43ms/step - loss: 1.2487 - accuracy: 0.6066 - val_loss: 1.5567 - val_accuracy: 0.5483
Epoch 10/10
31247/31247 [==============================] - 1453s 45ms/step - loss: 1.2460 - accuracy: 0.6074 - val_loss: 1.5500 - val_accuracy: 0.5485


In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2),
    model
])

shakespeare_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 lambda_3 (Lambda)           (None, None)              0         
                                                                 
 sequential (Sequential)     (None, None, 39)          61719     
                                                                 
Total params: 61,719
Trainable params: 61,719
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 0s 369ms/step


'e'

In [ ]:
!ls /content/my_shakespeare_model/variables//

variables.data-00000-of-00001  variables.index


In [ ]:

model_two = tf.keras.models.load_model("/content/models/my_shakespeare_model")

OSError: ignored

In [ ]:
shakespeare_model.save("sample_data/my_shakespeare_model")

# Generating fake Shakesperean text

In [ ]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)

In [ ]:
def next_char(text, temperature=1):
  y_proba = shakespeare_model.predict([text])[0, -1:]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
  return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extended_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text

In [ ]:
tf.random.set_seed(42)
print(extended_text("To be or not to be"), temperature=0.01)

In [ ]:
print(extended_text("To be or not to be"), temperature=1)

In [ ]:
print(extended_text("To be or not to be"), temperature=1)

# Stateful RNN